<a href="https://colab.research.google.com/github/wesslen/rag-workflow/blob/main/01_initial_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!uv pip install --system llama-index llama-index-embeddings-huggingface llama-index-llms-openai-like

In [ ]:
import os
from google.colab import userdata

API_KEY = userdata.get('API_KEY')
BASE_URL = userdata.get('BASE_URL')

In [ ]:
# take WF Code of Conduct
!wget -O text.txt https://r.jina.ai/https://www.wellsfargo.com/assets/pdf/about/corporate/code-of-conduct.pdf

In [ ]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model="/models/NousResearch/Meta-Llama-3.1-8B-Instruct", api_base=BASE_URL, api_key=API_KEY)
# response = llm.complete("Hello World!")
# print(str(response))

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

# Loading text files in root
documents = SimpleDirectoryReader(".").load_data()

Settings.chunk_size = 100
Settings.chunk_overlap = 20

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# llm=None - let's do retrieval only
rag_application = index.as_query_engine(similarity_top_k=5, llm=None)

In [ ]:
# An example input to your RAG application
user_input = "What are Wells Fargo's policies on lobbying and pay-to-play laws?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
response_object = rag_application.query(user_input)

# Process the response object to get the output string
# and retrieved nodes
if response_object is not None:
    actual_output = response_object.response
    retrieval_context = [node.get_content() for node in response_object.source_nodes]

In [ ]:
actual_output

In [ ]:
retrieval_context